## Get Data

Here we will merge two datasets: BoxOfficeMojo (boxofficemojo.com) and The Movie Database (TMDB). Here is what each contains:

| Feature / Data Point                                  | **Box Office Mojo (BOM)** 🏛               | **TMDb (The Movie Database)** 🎬                       |
| ----------------------------------------------------- | ------------------------------------------ | ------------------------------------------------------ |
| **Domestic box office grosses**                       | ✅ Accurate (daily/weekly/yearly US/Canada) | ❌ Not provided (only worldwide revenue, often missing) |
| **Worldwide box office**                              | ⚠️ Limited / inconsistent                  | ✅ Available (but often incomplete)                     |
| **Theater counts**                                    | ✅ Number of theaters per release           | ❌ Not available                                        |
| **Budgets**                                           | ❌ Rarely included                          | ✅ Included (when known)                                |
| **Genres**                                            | ❌ Not available                            | ✅ Rich genre metadata (IDs + names)                    |
| **Languages**                                         | ❌ Not available                            | ✅ Original language + spoken languages                 |
| **Production companies/countries**                    | ❌ Not available                            | ✅ Provided                                             |
| **Movie metadata** (runtime, overview, posters, etc.) | ❌ Not available                            | ✅ Extensive metadata                                   |
| **Upcoming movies (future years)**                    | ❌ Only past releases                       | ✅ Includes “In Production”, “Planned”, 2026+           |
| **Filters** (e.g. exclude TV, docs, non-English)      | ❌ No filters                               | ✅ Yes (via metadata)                                   |
| **Coverage**                                          | ✅ US theatrical releases only              | ✅ Worldwide releases (films + TV)                      |


### Setup

In [1]:
# ---- 0) Setup / imports (install if missing) ----
import subprocess, sys, os, time, json, requests, re
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

import sys, os
from pathlib import Path

# make sure the current folder (code/) is on sys.path
sys.path.insert(0, str(Path.cwd()))

import importlib, movie_lists
print(movie_lists.__file__) 
importlib.reload(movie_lists)        

from movie_lists import (
    MARVEL_MCU_FILMS,
    DC_FILMS,
    STAR_WARS_FILMS,
    FAST_FURIOUS_FILMS,
    WIZARDING_WORLD_FILMS,
    ALL_LIVE_ACTION_REMAKES,
    MEDIA_ADAPTATIONS,
    ALL_SUPERHERO_FILMS,
    REMAKE_PATTERNS,
    REMAKE_TITLE_INDICATORS,
    FRANCHISE_SEQUELS,
    apply_title_corrections,
    normalize_domestic_titles
)

def _ensure(pkg, import_name=None):
    try:
        __import__(import_name or pkg.replace("-", "_"))
        print(f"✅ {pkg} already installed")
    except Exception:
        print(f"📦 Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        print(f"✅ {pkg} installed")

for pkg in ["requests", "beautifulsoup4"]:
    _ensure(pkg)


/Users/jasmineplows/Documents/California/Projects/box_office/code/movie_lists.py
✅ requests already installed
📦 Installing beautifulsoup4...
✅ beautifulsoup4 installed


In [2]:
# ---- 1) Globals ----
DATA_DIR = "../data"
os.makedirs(DATA_DIR, exist_ok=True)

START_YEAR = 2010
END_YEAR = 2026
FORCE_REFRESH = False   # flip to True to re-scrape/re-fetch



In [3]:
# ---- 2) Cache wrapper - so you don't have to rescrape every time ----
def load_or_build_csv(path, builder_fn, *, force=FORCE_REFRESH, name="dataset"):
    """
    If `path` exists and not forcing, load CSV.
    Otherwise, call `builder_fn()` -> DataFrame, save to CSV, return it.
    """
    try:
        if (not force) and os.path.exists(path) and os.path.getsize(path) > 0:
            print(f"🗂️  Using cached {name}: {os.path.relpath(path)}")
            return pd.read_csv(path)
    except Exception as e:
        print(f"⚠️  Cache read issue for {name}: {e} — rebuilding")

    print(f"🔄 Building {name} …")
    df = builder_fn()
    os.makedirs(os.path.dirname(path), exist_ok=True)
    df.to_csv(path, index=False)
    print(f"💾 Saved {name} → {os.path.relpath(path)}  ({len(df)} rows)")
    return df


### Box Office Mojo Scrape Function

In [4]:
# ---- 3) Fetch All-Time Domestic Grosses (lifetime) + distributors (integrated) ----
import time, warnings
from bs4 import BeautifulSoup

def _norm_title_local(title: str) -> str:
    if pd.isna(title): return title
    t = str(title).strip()
    for ep in ["Episode I - ","Episode II - ","Episode III - ",
               "Episode IV - ","Episode V - ","Episode VI - ",
               "Episode VII - ","Episode VIII - ","Episode IX - "]:
        t = t.replace(ep, "")
    t = t.replace(" & ", " and ")
    return " ".join(t.split())

def _fetch_bom_distributors_for_year(year, max_pages=50, per_page=200, sleep=0.35, retries=3, retry_sleep=1.0):
    base = f"https://www.boxofficemojo.com/year/{year}/"
    headers = {
        "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/127.0.0.0 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Connection": "keep-alive",
    }
    frames, offset = [], 0
    for _ in range(max_pages):
        url = base if offset == 0 else f"{base}?offset={offset}"

        html = None
        last_exc = None
        for _try in range(retries):
            try:
                r = requests.get(url, headers=headers, timeout=20)
                if r.status_code == 200 and ("mojo-body-table" in r.text or "<table" in r.text):
                    html = r.text
                    break
                last_exc = RuntimeError(f"HTTP {r.status_code} / unexpected content")
            except Exception as e:
                last_exc = e
            time.sleep(retry_sleep)
        if html is None:
            break

        soup = BeautifulSoup(html, "html.parser")
        table = soup.select_one("table.a-bordered.a-horizontal-stripes.mojo-body-table")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            if table is not None:
                tables = pd.read_html(str(table))
            else:
                tables = pd.read_html(html)

        # table with Distributor + Release/Release Group
        candidates = [t for t in tables if ("Distributor" in t.columns and
                                            (("Release" in t.columns) or ("Release Group" in t.columns)))]
        if not candidates:
            break
        df = candidates[0].copy()
        if df.empty:
            break

        df["title"] = df["Release"] if "Release" in df.columns else df["Release Group"]
        df["release_year"] = year
        df = df[["title", "release_year", "Distributor"]].rename(columns={"Distributor": "distributor"})
        frames.append(df)

        if len(df) < per_page:
            break
        offset += per_page
        time.sleep(sleep)

    out = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["title","release_year","distributor"])
    out["title_normalized"] = out["title"].astype(str).apply(_norm_title_local)
    out = (out.sort_values(["release_year","distributor"])
             .drop_duplicates(subset=["title_normalized","release_year"], keep="first"))
    return out[["title_normalized","release_year","distributor"]]

def fetch_alltime_domestic(max_pages=50, sleep=0.35, per_page=200, retries=3, retry_sleep=1.0):
    """
    Scrape All-Time Domestic lifetime list AND attach distributor (via year pages for START_YEAR..END_YEAR).
    Returns columns: ['title','domestic_revenue','release_year','rank','distributor']
    """
    endpoints = [
        "https://www.boxofficemojo.com/chart/domestic/",
        "https://www.boxofficemojo.com/chart/top_lifetime_gross/",
        "https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=NA",
    ]
    headers = {
        "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/127.0.0.0 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Connection": "keep-alive",
    }

    frames, used_base = [], None
    for base in endpoints:
        frames.clear()
        used_base = base
        offset = 0
        for _ in range(max_pages):
            url = f"{base}&offset={offset}" if ("?" in base and offset) else (f"{base}?offset={offset}" if offset else base)
            html, last_exc = None, None
            for _try in range(retries):
                try:
                    r = requests.get(url, headers=headers, timeout=20)
                    if r.status_code == 200 and r.text and ("mojo-body-table" in r.text or "<table" in r.text):
                        html = r.text
                        break
                    last_exc = RuntimeError(f"HTTP {r.status_code}")
                except Exception as e:
                    last_exc = e
                time.sleep(retry_sleep)
            if html is None:
                if offset == 0:
                    frames.clear()
                break

            soup = BeautifulSoup(html, "html.parser")
            table = soup.select_one("table.a-bordered.a-horizontal-stripes.mojo-body-table")
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)
                tables = pd.read_html(str(table) if table is not None else html)

            def _norm_cols(df): return [str(c).strip() for c in df.columns]
            candidates = []
            for t in tables:
                t.columns = _norm_cols(t)
                cols = {c.lower() for c in t.columns}
                if "title" in cols and ("lifetime gross" in cols or "gross" in cols) and ("year" in cols or "release year" in cols or "rank" in cols):
                    candidates.append(t)
            if not candidates:
                break

            df = candidates[0]
            if df.empty:
                break
            frames.append(df)

            if len(df) < per_page:
                break
            offset += per_page
            time.sleep(sleep)

        if frames:
            break

    if not frames:
        raise RuntimeError("No data scraped from Box Office Mojo across all endpoints")

    alltime = pd.concat(frames, ignore_index=True)
    rename_candidates = {
        "Title": "title",
        "Lifetime Gross": "domestic_revenue",
        "Gross": "domestic_revenue",
        "Year": "release_year",
        "Release Year": "release_year",
        "Rank": "rank",
    }
    alltime.columns = [str(c).strip() for c in alltime.columns]
    for k, v in list(rename_candidates.items()):
        if k in alltime.columns:
            alltime = alltime.rename(columns={k: v})

    for col in ["title", "domestic_revenue", "release_year", "rank"]:
        if col not in alltime.columns:
            alltime[col] = np.nan

    alltime["title"] = alltime["title"].astype(str).str.strip()
    alltime["domestic_revenue"] = pd.to_numeric(alltime["domestic_revenue"].astype(str).str.replace(r"[\$,]", "", regex=True), errors="coerce")
    alltime["release_year"] = pd.to_numeric(alltime["release_year"], errors="coerce").astype("Int64")
    alltime["rank"] = pd.to_numeric(alltime["rank"], errors="coerce").astype("Int64")
    alltime = alltime.dropna(subset=["title", "domestic_revenue"], how="any")

    # ALWAYS attach distributors for your modeling window
    y0 = int(START_YEAR)
    y1 = int(END_YEAR)
    print(f"  • Attaching distributors from BOM year pages ({y0}–{y1}) …")
    dist_idx = []
    for y in range(y0, y1 + 1):
        dfy = _fetch_bom_distributors_for_year(y)
        if not dfy.empty:
            dist_idx.append(dfy)
    if dist_idx:
        dist_idx = pd.concat(dist_idx, ignore_index=True)
        alltime["title_normalized"] = alltime["title"].apply(_norm_title_local)
        alltime = alltime.merge(dist_idx, on=["title_normalized","release_year"], how="left")
        alltime = alltime.drop(columns=["title_normalized"])
    else:
        alltime["distributor"] = np.nan

    print(f"📊 All-Time Domestic dataset ready from {used_base} : {alltime.shape}")
    return alltime


### TMDB Scrape Function

In [5]:
# ---- 4) Fetch TMDb (v3/v4) ----
def fetch_tmdb_movies(
    api_key,
    start_year=2010,
    end_year=2026,
    include_upcoming_pass=True,   # kept for signature compatibility (not used by TMDb discover directly)
    max_pages_per_year=5,
    region_us=True,
    min_vote_count=0,
    sleep_sec=0.25
):
    """
    Fetch movies from TMDb API (v3 or v4 auth).

    - If api_key looks like a JWT (starts with 'eyJ'), uses v4 Bearer header.
    - Otherwise, treats it as a v3 key and passes ?api_key=...
    - For each discovered movie, fetches detail to attach 'production_countries'.
    - Returns a pandas DataFrame with optional 'release_year' column.
    """
    if not api_key or api_key == "YOUR_TMDB_API_KEY_HERE":
        raise RuntimeError("TMDb API key missing — add it to config.json or env")

    is_v4 = str(api_key).startswith("eyJ")  # v4 tokens look like JWTs
    base_url = "https://api.themoviedb.org/3"

    headers = {"accept": "application/json"}
    if is_v4:
        headers["Authorization"] = f"Bearer {api_key}"

    all_movies = []

    for year in range(int(start_year), int(end_year) + 1):
        for page in range(1, int(max_pages_per_year) + 1):
            params = {
                "primary_release_year": year,
                "page": page,
                "language": "en-US",
                "include_adult": "false",
                "primary_release_date.gte": f"{year}-01-01",
                "primary_release_date.lte": f"{year}-12-31",
                "with_release_type": "2|3|4|5"
            }
            if region_us:
                params["region"] = "US"
            if min_vote_count > 0:
                params["vote_count.gte"] = int(min_vote_count)
            if not is_v4:  # v3 key
                params["api_key"] = api_key

            try:
                r = requests.get(
                    f"{base_url}/discover/movie",
                    headers=headers,
                    params=params,
                    timeout=20
                )
                if r.status_code != 200:
                    print(f"    Error {r.status_code} year={year} page={page}: {r.text[:200]}")
                    break

                data = r.json()
                results = data.get("results", [])
                total_pages = data.get("total_pages", 1) or 1

                for payload in results:
                    movie_id = payload.get("id")
                    if movie_id is None:
                        # Keep the raw payload if for some reason id is missing
                        all_movies.append(payload)
                        continue

                    # Fetch detail to attach production_countries
                    detail_url = f"{base_url}/movie/{movie_id}"
                    detail_params = {} if is_v4 else {"api_key": api_key}
                    detail_countries = []
                    try:
                        detail_resp = requests.get(
                            detail_url,
                            headers=headers,
                            params=detail_params,
                            timeout=20
                        )
                        if detail_resp.ok:
                            detail_data = detail_resp.json()
                            detail_countries = detail_data.get("production_countries", []) or []
                    except Exception as detail_exc:
                        print(f"       detail fetch failed for {movie_id}: {detail_exc}")

                    payload["production_countries"] = detail_countries
                    all_movies.append(payload)

                if page >= total_pages:
                    break

            except Exception as e:
                print(f"    Request failed year={year} page={page}: {e}")
                break

            time.sleep(float(sleep_sec))

    if not all_movies:
        raise RuntimeError("No movies fetched from TMDb")

    df = pd.DataFrame(all_movies)
    if "release_date" in df.columns:
        df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
        df["release_year"] = df["release_date"].dt.year
    return df


### Run

Note you should have a config.json file with TMDB_API_KEY specified

In [6]:
# ---- 5) Load or build datasets with cache (BOM now includes distributor) ----
ALLTIME_CSV = os.path.join(DATA_DIR, "boxoffice_alltime_domestic.csv")
TMDB_CSV   = os.path.join(DATA_DIR, "tmdb_filtered.csv")

domestic_df = load_or_build_csv(
    ALLTIME_CSV,
    builder_fn=fetch_alltime_domestic,
    name="All-Time Domestic (lifetime + distributors)"
)

def load_tmdb_key():
    key = None
    for up in ["", "..", "../..", "../../.."]:
        cfg_path = os.path.join(os.getcwd(), up, "config.json")
        if os.path.exists(cfg_path):
            try:
                with open(cfg_path, "r", encoding="utf-8") as f:
                    cfg = json.load(f)
                key = cfg.get("TMDB_V4_TOKEN") or cfg.get("TMDB_API_KEY")
                if key:
                    print(f"🔑 Loaded TMDb key from {cfg_path}")
                    break
            except Exception as e:
                print(f"⚠️ Could not parse {cfg_path}: {e}")
    if not key:
        key = os.getenv("TMDB_V4_TOKEN") or os.getenv("TMDB_API_KEY")
    if not key:
        raise RuntimeError("❌ No TMDb API key found in config.json or environment!")
    return key.strip().strip('"').strip("'")

def build_tmdb_filtered():
    TMDB_API_KEY = load_tmdb_key()
    return fetch_tmdb_movies(
        TMDB_API_KEY,
        start_year=START_YEAR,
        end_year=END_YEAR,
        include_upcoming_pass=False,
        max_pages_per_year=100,
        region_us=True,
        min_vote_count=0,
        sleep_sec=0.2
    )

tmdb_df = load_or_build_csv(
    TMDB_CSV,
    builder_fn=build_tmdb_filtered,
    name="TMDb (filtered)"
)

print(f"\n✅ Domestic: {domestic_df.shape}, TMDb: {tmdb_df.shape}")
display(domestic_df.head())
display(tmdb_df.head())


🗂️  Using cached All-Time Domestic (lifetime + distributors): ../data/boxoffice_alltime_domestic.csv
🗂️  Using cached TMDb (filtered): ../data/tmdb_filtered.csv

✅ Domestic: (10000, 5), TMDb: (32699, 16)


,rank,title,domestic_revenue,release_year,distributor
0,1,Star Wars: Episode VII - The Force Awakens,936662225,2015,Walt Disney Studios Motion Pictures
1,2,Avengers: Endgame,858373000,2019,Walt Disney Studios Motion Pictures
2,3,Spider-Man: No Way Home,814866759,2021,Sony Pictures Releasing
3,4,Avatar,785221649,2009,NaN
4,5,Top Gun: Maverick,718732821,2022,Paramount Pictures


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,production_countries,release_year
0,False,/gqby0RhyehP3uRrzmdyUZ0CgPPe.jpg,"[28, 878, 12]",27205,en,Inception,"Cobb, a skilled thief who commits corporate es...",28.4777,/ljsZTbVsrQSqZgWeep2B1QiDKuh.jpg,2010-07-16,Inception,False,8.370,37995,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010
1,False,/AqLcLsGGTzAjm3pCCq0CZCQrp6m.jpg,"[12, 14]",12444,en,Harry Potter and the Deathly Hallows: Part 1,"Harry, Ron and Hermione walk away from their l...",19.6889,/iGoXIpQb7Pot00EEdwpwPajheZ5.jpg,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,False,7.738,19751,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2010
2,False,/7lmBufEG7P7Y1HClYK3gCxYrkgS.jpg,"[12, 28, 878]",10138,en,Iron Man 2,With the world now aware of his dual life as t...,18.6595,/6WBeq4fCfn7AN0o21W9qNcRF2l9.jpg,2010-05-07,Iron Man 2,False,6.848,21737,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010
3,False,/cvybPlcv87ZqLpBi1OaartRka6.jpg,"[28, 12, 14, 27]",12685,en,BloodRayne: The Third Reich,Rayne fights against the Nazis in Europe durin...,17.4416,/kS48eBvmVXtNJGXWW5QMQmm1JbO.jpg,2010-06-10,BloodRayne: The Third Reich,False,3.816,234,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",2010
4,False,/oguoQWtQJcqIkmmUNbe3CmLx6AP.jpg,"[12, 14, 18, 10749]",24021,en,The Twilight Saga: Eclipse,Bella once again finds herself surrounded by d...,16.3754,/dK4Gi1UdMiHzHc7r7CZQG4IQ9Sr.jpg,2010-06-23,The Twilight Saga: Eclipse,False,6.203,8992,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010


In [7]:
# Production country info will be processed in notebook 2 (feature engineering)
# This keeps data collection and feature engineering properly separated
print('✅ Production countries preserved for feature engineering in notebook 2')

✅ Production countries preserved for feature engineering in notebook 2


### Merge datasets

In [8]:
# ============================================
# 6) Merge TMDb with All-Time Domestic (exact + fuzzy), carry distributor
# ============================================

# Install rapidfuzz if needed (for fuzzy merge fallback)
import subprocess, sys, warnings
try:
    from rapidfuzz import process, fuzz
except Exception:
    print("📦 Installing rapidfuzz…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rapidfuzz"])
    from rapidfuzz import process, fuzz

def normalize_title(title: str) -> str:
    if pd.isna(title): 
        return title
    t = str(title).strip()
    for ep in ["Episode I - ","Episode II - ","Episode III - ",
               "Episode IV - ","Episode V - ","Episode VI - ",
               "Episode VII - ","Episode VIII - ","Episode IX - "]:
        t = t.replace(ep, "")
    t = t.replace(" & ", " and ")
    return " ".join(t.split())

# ---- Clean/standardize Domestic (lifetime) ----
domestic_clean = domestic_df.copy()
for col in ["title", "release_year", "domestic_revenue", "rank", "distributor"]:
    if col not in domestic_clean.columns:
        domestic_clean[col] = np.nan

domestic_clean["title"] = domestic_clean["title"].astype(str).str.strip()
domestic_clean["title_normalized"] = domestic_clean["title"].apply(normalize_title)
domestic_clean["release_year"] = pd.to_numeric(domestic_clean["release_year"], errors="coerce").astype("Int64")
domestic_clean["domestic_revenue"] = pd.to_numeric(domestic_clean["domestic_revenue"], errors="coerce")
domestic_clean["distributor"] = domestic_clean["distributor"].astype(str).str.strip()

# Collapse to one row per (title_normalized, release_year)
domestic_keyed = (
    domestic_clean
    .dropna(subset=["title_normalized", "release_year"])
    .groupby(["title_normalized", "release_year"], as_index=False)
    .agg({
        "title": "first",
        "domestic_revenue": "max",
        "rank": "min",
        "distributor": "first"
    })
)

# ---- Clean/standardize TMDb ----
tmdb_clean = tmdb_df.copy()
if "release_date" in tmdb_clean.columns:
    tmdb_clean["release_date"] = pd.to_datetime(tmdb_clean["release_date"], errors="coerce")
    tmdb_clean["release_year"] = tmdb_clean["release_date"].dt.year.where(
        tmdb_clean.get("release_year").isna() if "release_year" in tmdb_clean.columns else True,
        tmdb_clean.get("release_year")
    )
tmdb_clean["release_year"] = pd.to_numeric(tmdb_clean["release_year"], errors="coerce").astype("Int64")

tmdb_clean["original_language"] = tmdb_clean.get("original_language").fillna("unknown")

if "genres" not in tmdb_clean.columns:
    if "genre_ids" in tmdb_clean.columns:
        tmdb_clean["genres"] = tmdb_clean["genre_ids"].astype(str)
    else:
        tmdb_clean["genres"] = ""

def _contains_genre_ids_as_text(s, ids=("99", "10770")):
    st = str(s);  return any(f"{gid}" in st for gid in ids)

tmdb_clean = tmdb_clean[~tmdb_clean["genres"].apply(_contains_genre_ids_as_text)].copy()

tmdb_clean["title"] = tmdb_clean["title"].astype(str).str.strip()
tmdb_clean["title_normalized"] = tmdb_clean["title"].apply(normalize_title)

tmdb_keyed = (
    tmdb_clean.sort_values(["release_year","vote_count","popularity"], ascending=[True, False, False])
              .drop_duplicates(subset=["title_normalized","release_year"], keep="first")
)

print("🔗 Exact merge TMDb ⟷ Domestic (lifetime)…")
merged_df = pd.merge(
    tmdb_keyed,
    domestic_keyed[["title_normalized","release_year","domestic_revenue","rank","distributor"]],
    on=["title_normalized","release_year"],
    how="left",
    suffixes=("", "_domestic"),
)
exact_hits = merged_df["domestic_revenue"].notna().sum()
print(f"✅ Exact matches: {exact_hits:,}")

# ---------- Fuzzy fallback (same-year only) ----------
def fuzzy_fill_domestic(merged, domestic, score_cutoff=90):
    dom_by_year = {}
    dom = domestic[["title_normalized", "release_year", "domestic_revenue", "rank", "distributor"]].dropna(subset=["title_normalized"])
    for y, sub in dom.groupby("release_year"):
        dom_by_year[int(y)] = (sub["title_normalized"].tolist(), sub.index.tolist())

    added = 0
    def _is_safe_match(query, candidate, score):
        candidate_text = str(candidate or "").strip()
        if not candidate_text:
            return False
        query_text = str(query or "").strip()
        if not query_text:
            return False
        if score < max(score_cutoff, 93):
            return False
        len_q = len(query_text)
        len_c = len(candidate_text)
        max_len = max(len_q, len_c)
        if max_len:
            length_gap = abs(len_q - len_c)
            if length_gap > max(3, int(0.35 * max_len)):
                return False
        tokens_q = set(re.findall(r"\w+", query_text.lower()))
        tokens_c = set(re.findall(r"\w+", candidate_text.lower()))
        if tokens_q and tokens_c:
            overlap = len(tokens_q & tokens_c) / max(len(tokens_q), len(tokens_c))
            if overlap < 0.6:
                return False
        return True
    missing_mask = merged["domestic_revenue"].isna()
    groups = merged[missing_mask].groupby("release_year").groups

    for y, idxs in groups.items():
        if pd.isna(y): continue
        y = int(y)
        if y not in dom_by_year: continue
        titles_dom, idxs_dom = dom_by_year[y]
        if not titles_dom: continue

        for ridx in idxs:
            q = merged.at[ridx, "title_normalized"]
            if not isinstance(q, str) or not q: continue
            match = process.extractOne(q, titles_dom, scorer=fuzz.WRatio, score_cutoff=90)
            if not match: continue
            candidate, score, pos = match
            if not _is_safe_match(q, candidate, score):
                continue
            dom_idx = idxs_dom[pos]
            merged.at[ridx, "domestic_revenue"] = domestic.loc[dom_idx, "domestic_revenue"]
            merged.at[ridx, "rank"] = domestic.loc[dom_idx, "rank"]
            merged.at[ridx, "distributor"] = domestic.loc[dom_idx, "distributor"]
            added += 1
    return merged, added

print("🧪 Fuzzy matching unmatched rows (same year)…")
merged_df, fuzzy_added = fuzzy_fill_domestic(merged_df, domestic_keyed, score_cutoff=90)
print(f"➕ Fuzzy matches added: {fuzzy_added:,}")

# Final safety: drop any remaining duplicates on (title, release_year) keeping highest domestic
if merged_df.duplicated(subset=["title","release_year"], keep=False).any():
    merged_df = (merged_df
        .sort_values(["release_year","domestic_revenue"], ascending=[True, False])
        .drop_duplicates(subset=["title","release_year"], keep="first"))

# Canonical revenue columns
merged_df["revenue_domestic"] = pd.to_numeric(merged_df["domestic_revenue"], errors="coerce")
merged_df["revenue"] = merged_df["revenue_domestic"]  # modeling target = lifetime domestic

print(f"✅ Merge complete. Rows: {len(merged_df):,}")
display(merged_df.head(5))


🔗 Exact merge TMDb ⟷ Domestic (lifetime)…
✅ Exact matches: 2,243
🧪 Fuzzy matching unmatched rows (same year)…
➕ Fuzzy matches added: 48
✅ Merge complete. Rows: 28,347


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,production_countries,release_year,genres,title_normalized,domestic_revenue,rank,distributor,revenue_domestic,revenue
0,False,/sb3kfTJpbyHoA43apCJOvp2dLUa.jpg,"[28, 16, 878, 18]",408145,ja,機動戦士ガンダムSEED スペシャルエディション II 遥かなる暁,"The battle lines have been drawn, placing old ...",3.8325,/fHQxkmtWFb3C9tLD4nj2MfOTxTf.jpg,2005-09-13,Mobile Suit Gundam SEED: Special Edition II - ...,False,5.300,3,"[{'iso_3166_1': 'JP', 'name': 'Japan'}]",2005,"[28, 16, 878, 18]",Mobile Suit Gundam SEED: Special Edition II - ...,NaN,NaN,NaN,NaN,NaN
1,False,/gqby0RhyehP3uRrzmdyUZ0CgPPe.jpg,"[28, 878, 12]",27205,en,Inception,"Cobb, a skilled thief who commits corporate es...",28.4777,/ljsZTbVsrQSqZgWeep2B1QiDKuh.jpg,2010-07-16,Inception,False,8.370,37995,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010,"[28, 878, 12]",Inception,292587330.0,120.0,Warner Bros.,292587330.0,292587330.0
2,False,/rbZvGN1A1QyZuoKzhCw8QPmf2q0.jpg,"[18, 53, 9648]",11324,en,Shutter Island,World War II soldier-turned-U.S. Marshal Teddy...,14.0080,/nrmXQ0zcZUL8jFLrakWc90IR8z9.jpg,2010-02-19,Shutter Island,False,8.201,24901,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010,"[18, 53, 9648]",Shutter Island,128012934.0,570.0,Paramount Pictures,128012934.0,128012934.0
3,False,/7lmBufEG7P7Y1HClYK3gCxYrkgS.jpg,"[12, 28, 878]",10138,en,Iron Man 2,With the world now aware of his dual life as t...,18.6595,/6WBeq4fCfn7AN0o21W9qNcRF2l9.jpg,2010-05-07,Iron Man 2,False,6.848,21737,"[{'iso_3166_1': 'US', 'name': 'United States o...",2010,"[12, 28, 878]",Iron Man 2,312433331.0,104.0,Paramount Pictures,312433331.0,312433331.0
4,False,/AqLcLsGGTzAjm3pCCq0CZCQrp6m.jpg,"[12, 14]",12444,en,Harry Potter and the Deathly Hallows: Part 1,"Harry, Ron and Hermione walk away from their l...",19.6889,/iGoXIpQb7Pot00EEdwpwPajheZ5.jpg,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,False,7.738,19751,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2010,"[12, 14]",Harry Potter and the Deathly Hallows: Part 1,296374621.0,115.0,Warner Bros.,296374621.0,296374621.0


In [9]:

# Alias cleanup + duplicate handling before title corrections
print("🔎 Checking for duplicate entries and known aliases…")

# Normalise Twisters aliases before dedupe
alias_map = {
    'The Twisters': 'Twisters',
    'The Twisters (2024)': 'Twisters',
}
for alias, canonical in alias_map.items():
    mask = merged_df['title'].eq(alias)
    if mask.any():
        hits = int(mask.sum())
        print(f"   • Renaming {hits} occurrence(s) of '{alias}' → '{canonical}'")
        merged_df.loc[mask, 'title'] = canonical

# Drop exact duplicates (same title/year/revenue)
before_exact = len(merged_df)
merged_df = merged_df.drop_duplicates(subset=['title', 'release_year', 'revenue_domestic']).copy()
removed_exact = before_exact - len(merged_df)
if removed_exact:
    print(f"   • Removed {removed_exact} exact duplicate row(s)")
else:
    print("   • No exact duplicate rows found")

# For residual collisions on (title, year), keep the highest revenue observation
dup_mask = merged_df.duplicated(subset=['title', 'release_year'], keep=False)
if dup_mask.any():
    conflicts = (
        merged_df.loc[dup_mask, ['title', 'release_year', 'revenue_domestic']]
        .sort_values(['title', 'release_year', 'revenue_domestic'], ascending=[True, True, False])
    )
    print("   • Conflicting rows detected (same title/year, differing revenue):")
    display(conflicts)
    merged_df = (
        merged_df.sort_values(['title', 'release_year', 'revenue_domestic'], ascending=[True, True, False])
        .drop_duplicates(subset=['title', 'release_year'], keep='first')
        .reset_index(drop=True)
    )
    print("   • Retained the highest-revenue entry for each title/year combination")
else:
    print("   • No title/year revenue conflicts detected")

# Apply systematic title corrections (handles Deadpool 3 → Deadpool & Wolverine, etc.)
merged_df = apply_title_corrections(merged_df)


🔎 Checking for duplicate entries and known aliases…
   • Renaming 1 occurrence(s) of 'The Twisters' → 'Twisters'
   • No exact duplicate rows found
   • Conflicting rows detected (same title/year, differing revenue):


,title,release_year,revenue_domestic
24049,Twisters,2024,267762265.0
24410,Twisters,2024,NaN


   • Retained the highest-revenue entry for each title/year combination
✅ Applied 3 title corrections


### Filter + Export
Now, go to `2_feature_engineering.ipynb`

In [10]:
# ============================================
# 7) Final filters + BOM 2026 calendar integration + save + summary
# ============================================

final_df = merged_df.copy()

# Modeling window (2010-2026)
final_df = final_df[final_df["release_year"].between(START_YEAR, END_YEAR)]

# For movies before 2026: must have lifetime domestic revenue
# For 2026 movies: include them even without revenue data (since they haven't been released yet)
pre_2026_mask = final_df["release_year"] < 2026
year_2026_mask = final_df["release_year"] == 2026

final_df = final_df[
    (pre_2026_mask & final_df["revenue_domestic"].notna() & (final_df["revenue_domestic"] > 0)) |
    year_2026_mask
]

# Drop TV/docs leftovers
if "genres" in final_df.columns:
    final_df = final_df[
        ~final_df["genres"].astype(str).str.contains("99|10770|Documentary|TV", case=False, na=False)
    ]

final_df = normalize_domestic_titles(final_df)

# ============================================
# 7.5) FILTER 2026 TO BOX OFFICE MOJO CALENDAR + ASSIGN DISTRIBUTORS
# ============================================

print("🎬 Filtering 2026 movies to Box Office Mojo calendar releases...")

# Box Office Mojo 2026 calendar movies with official distributors
# Compiled from https://www.boxofficemojo.com/calendar/2026-XX-01/ pages
# Updated to include missing movies like The Super Mario Galaxy Movie
bom_2026_movies = {
    # Major Studio Releases
    'The Mandalorian and Grogu': 'Walt Disney Studios Motion Pictures',
    'Avengers: Doomsday': 'Walt Disney Studios Motion Pictures',
    'Spider-Man: Brand New Day': 'Sony Pictures Releasing',
    'Toy Story 5': 'Walt Disney Studios Motion Pictures',
    'Moana': 'Walt Disney Studios Motion Pictures',
    'Hoppers': 'Walt Disney Studios Motion Pictures',
    'The Devil Wears Prada 2': 'Walt Disney Studios Motion Pictures',
    'Send Help': 'Walt Disney Studios Motion Pictures',
    'The Dog Stars': 'Walt Disney Studios Motion Pictures',
    'Untitled Disney': 'Walt Disney Studios Motion Pictures',
    'Untitled Taika Waititi Star Wars Film': 'Walt Disney Studios Motion Pictures',
    
    'Jumanji 3': 'Sony Pictures Releasing',
    '28 Years Later: The Bone Temple': 'Sony Pictures Releasing',
    'GOAT': 'Sony Pictures Releasing',
    'The Breadwinner': 'Sony Pictures Releasing',
    'Animal Friends': 'Sony Pictures Releasing',
    'Shiver': 'Sony Pictures Releasing',
    'A Private Life': 'Sony Pictures Releasing',
    
    'Mortal Kombat II': 'Warner Bros.',
    'Supergirl': 'Warner Bros.',
    'Cut Off': 'Warner Bros.',
    'Evil Dead Burn': 'Warner Bros.',
    'The Cat in the Hat': 'Warner Bros.',
    'Wuthering Heights': 'Warner Bros.',
    
    'The Odyssey': 'Universal Pictures',
    'Minions 3': 'Universal Pictures',
    'Violent Night 2': 'Universal Pictures',
    'Reminders of Him': 'Universal Pictures',
    'Untitled Steven Spielberg Amblin Universal Event Film': 'Universal Pictures',
    
    # Nintendo/Universal Movies
    'The Super Mario Galaxy Movie': 'Universal Pictures',
    'The Super Mario Bros. Movie 2': 'Universal Pictures',
    
    'Scream 7': 'Paramount Pictures',
    'Scary Movie 6': 'Paramount Pictures',
    'Primate': 'Paramount Pictures',
    'The Angry Birds Movie 3': 'Paramount Pictures',
    'Untitled Trey Parker / Matt Stone / Kendrick Lamar / Dave Free': 'Paramount Pictures',
    
    # Mid-size/Independent Distributors
    'Greenland: Migration': 'Lionsgate',
    'I Can Only Imagine 2': 'Lionsgate',
    'The Hunger Games: Sunrise on the Reaping': 'Lionsgate',
    'Mutiny': 'Lionsgate',
    
    'Mercy': 'Amazon Studios',
    'Is God Is': 'Amazon Studios',
    'Project Hail Mary': 'Amazon Studios',
    'Three Bags Full: A Sheep Detective Movie': 'Amazon Studios',
    
    'Psycho Killer': '20th Century Studios',
    'The Plague': 'IFC Films',
    'Night Patrol': 'IFC Films',
    'Charlie the Wonderdog': 'Viva Pictures',
    'The Pout-Pout Fish': 'Viva Pictures',
    'Return to Silent Hill': 'Iconic Events Releasing',
    "Good Luck, Have Fun, Don't Die": 'Briarcliff Entertainment',
    'Dracula: A Love Tale': 'Vertical Entertainment',
    'Mimics': 'ArtAffects Entertainment',
    'Solo Mio': 'Angel',
    'I Was A Stranger': 'Angel',
    'The Bride': 'IMAX',
    'Broken Bird': 'Independent',
}

# Filter 2026 movies to only include those in BOM calendar
movies_2026 = final_df[final_df['release_year'] == 2026]
bom_calendar_titles = set(bom_2026_movies.keys())

def normalize_title_for_matching(title):
    if pd.isna(title):
        return ''
    title = str(title).lower().strip()
    title = title.replace('the ', '').replace('a ', '').replace('an ', '')
    import re
    title = re.sub(r'[^a-z0-9\\s]', '', title)
    return ' '.join(title.split())

# Find matches between our 2026 movies and BOM calendar
matched_2026_indices = []
assigned_distributors = 0

for bom_title, distributor in bom_2026_movies.items():
    bom_normalized = normalize_title_for_matching(bom_title)
    
    # Look for matches in our dataset
    for idx, movie in movies_2026.iterrows():
        movie_normalized = normalize_title_for_matching(movie['title'])
        
        # Exact match or close match
        if (movie_normalized == bom_normalized or 
            bom_normalized in movie_normalized or 
            movie_normalized in bom_normalized):
            
            matched_2026_indices.append(idx)
            final_df.loc[idx, 'distributor'] = distributor
            assigned_distributors += 1
            break

# Keep only matched 2026 movies
if matched_2026_indices:
    # Get pre-2026 movies and matched 2026 movies
    pre_2026_movies = final_df[final_df['release_year'] < 2026]
    matched_2026_movies = final_df.loc[matched_2026_indices]
    
    # Combine them
    final_df = pd.concat([pre_2026_movies, matched_2026_movies], ignore_index=True)
    
    print(f"   ✓ Filtered to {len(matched_2026_movies)} BOM calendar 2026 movies")
    print(f"   ✓ Assigned {assigned_distributors} distributor assignments")
else:
    # If no matches found, remove all 2026 movies
    final_df = final_df[final_df['release_year'] < 2026]
    print("   ⚠️ No BOM calendar matches found - removed all 2026 movies")

# Save
MERGED_CSV = os.path.join(DATA_DIR, "dataset_domestic_lifetime_merged.csv")
final_df.to_csv(MERGED_CSV, index=False)
print(f"💾 Saved merged dataset → {MERGED_CSV}")

# Summary
print(f"Total movies: {len(final_df):,}")
if len(final_df) > 0:
    yr_min = int(final_df["release_year"].min())
    yr_max = int(final_df["release_year"].max())
    
    # Summary for pre-2026 movies (with revenue data)
    pre_2026_df = final_df[final_df["release_year"] < 2026]
    if len(pre_2026_df) > 0:
        rev_min = pre_2026_df["revenue_domestic"].min()
        rev_max = pre_2026_df["revenue_domestic"].max()
        rev_avg = pre_2026_df["revenue_domestic"].mean()
        print(f"Year range: {yr_min}–{yr_max}")
        print(f"Pre-2026 movies with revenue: {len(pre_2026_df):,}")
        print(f"Lifetime domestic range: ${rev_min:,.0f} — ${rev_max:,.0f}")
        print(f"Average lifetime domestic: ${rev_avg:,.0f}")
    
    # Summary for 2026 movies (BOM calendar only)
    year_2026_df = final_df[final_df["release_year"] == 2026]
    if len(year_2026_df) > 0:
        print(f"2026 BOM calendar movies: {len(year_2026_df):,}")
        
        # Show 2026 distributor breakdown
        dist_counts_2026 = year_2026_df['distributor'].value_counts()
        print(f"2026 distributor breakdown:")
        for dist, count in dist_counts_2026.items():
            pct = count / len(year_2026_df) * 100
            print(f"   {dist}: {count} movies ({pct:.1f}%)")

    cols = ["title", "release_year", "distributor", "revenue_domestic"]
    cols = [c for c in cols if c in final_df.columns]
    
    # Show top movies with revenue data
    movies_with_revenue = final_df[final_df["revenue_domestic"].notna()]
    if len(movies_with_revenue) > 0:
        print("\\nTop 10 movies by domestic revenue:")
        display(movies_with_revenue.nlargest(10, "revenue_domestic")[cols].reset_index(drop=True))
    
    # Show 2026 BOM calendar movies
    if len(year_2026_df) > 0:
        print(f"\\n2026 Box Office Mojo calendar movies:")
        sample_cols = ["title", "distributor", "popularity"]
        sample_cols = [c for c in sample_cols if c in year_2026_df.columns]
        display(year_2026_df[sample_cols].reset_index(drop=True))
else:
    print("⚠️ Final dataset is empty — review filters and merge keys.")

No title corrections needed
🎬 Filtering 2026 movies to Box Office Mojo calendar releases...
   ✓ Filtered to 57 BOM calendar 2026 movies
   ✓ Assigned 57 distributor assignments
💾 Saved merged dataset → ../data/dataset_domestic_lifetime_merged.csv
Total movies: 2,348
Year range: 2010–2026
Pre-2026 movies with revenue: 2,291
Lifetime domestic range: $514,870 — $936,662,225
Average lifetime domestic: $58,095,424
2026 BOM calendar movies: 57
2026 distributor breakdown:
   Angel: 11 movies (19.3%)
   Walt Disney Studios Motion Pictures: 8 movies (14.0%)
   Sony Pictures Releasing: 7 movies (12.3%)
   Warner Bros.: 6 movies (10.5%)
   Universal Pictures: 6 movies (10.5%)
   Paramount Pictures: 4 movies (7.0%)
   Amazon Studios: 4 movies (7.0%)
   Lionsgate: 3 movies (5.3%)
   Viva Pictures: 2 movies (3.5%)
   20th Century Studios: 1 movies (1.8%)
   IFC Films: 1 movies (1.8%)
   Iconic Events Releasing: 1 movies (1.8%)
   Briarcliff Entertainment: 1 movies (1.8%)
   IMAX: 1 movies (1.8%)
  

,title,release_year,distributor,revenue_domestic
0,Star Wars: The Force Awakens,2015,Walt Disney Studios Motion Pictures,936662225.0
1,Avengers: Endgame,2019,Walt Disney Studios Motion Pictures,858373000.0
2,Spider-Man: No Way Home,2021,Sony Pictures Releasing,814866759.0
3,Top Gun: Maverick,2022,Paramount Pictures,718732821.0
4,Black Panther,2018,Walt Disney Studios Motion Pictures,700426566.0
5,Avatar: The Way of Water,2022,20th Century Studios,684075767.0
6,Avengers: Infinity War,2018,Walt Disney Studios Motion Pictures,678815482.0
7,Jurassic World,2015,Universal Pictures,653406625.0
8,Inside Out 2,2024,Walt Disney Studios Motion Pictures,652980194.0
9,Deadpool & Wolverine,2024,Walt Disney Studios Motion Pictures,636745858.0


\n2026 Box Office Mojo calendar movies:


,title,distributor,popularity
0,The Mandalorian and Grogu,Walt Disney Studios Motion Pictures,13.8163
1,Avengers: Doomsday,Walt Disney Studios Motion Pictures,13.2883
2,Spider-Man: Brand New Day,Sony Pictures Releasing,6.5808
3,Toy Story 5,Walt Disney Studios Motion Pictures,4.1557
4,Moana,Walt Disney Studios Motion Pictures,4.0352
5,Hoppers,Walt Disney Studios Motion Pictures,4.4302
6,The Devil Wears Prada 2,Walt Disney Studios Motion Pictures,3.4087
7,Send Help,Walt Disney Studios Motion Pictures,2.7242
8,The Dog Stars,Walt Disney Studios Motion Pictures,3.0235
9,Добрый доктор,Angel,0.4465


The above table should match the screenshot below (visually remove movies from pre-2025)

![Box Office Mojo as of 09/21/2025](../box-office-mojo.png)
